# Beaderstadt Final: Regression Analysis<br>
**Author:** Alissa Beaderstadt<br>
**Date:** November 18, 2025<br>

## Introduction
In this project, I explore the Medical Costs Dataset from Kaggle.

## Imports
Import the necessary Python libraries for this notebook.

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.colors import ListedColormap
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

# Models
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

# Evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

## Section 1. Import and Inspect the Data
### 1.1 Load the Medical Costs Dataset
Load the dataset and display the first 10 rows.

In [3]:
# TODO

### 1.2 Check for Missing Values and Display Summary Statistics

In [4]:
# TODO

**Reflection 1:**
- What do you notice about the dataset?
  
-  Are there any data issues?


## Section 2. Data Exploration and Preparation
### 2.1 Explore data patterns and distributions
- Create histograms, boxplots, and count plots for categorical variables (as applicable).  
- Identify patterns, outliers, and anomalies in feature distributions.  
- I’ll also Check for class imbalance in the target variable (as applicable).

In [5]:
# TODO

### 2.2 Handle missing values and clean data
- Impute or drop missing values (as applicable).
- Remove or transform outliers (as applicable).
- Convert categorical data to numerical format using encoding (as applicable).

In [6]:
# TODO

### 2.3 Feature selection and engineering
- Create new features (as applicable).
- Transform or combine existing features to improve model performance (as applicable).
- Scale or normalize data (as applicable).

In [7]:
# TODO

**Reflection 2:**
- What patterns or anomalies do you see? 
- Do any features stand out? What preprocessing steps were necessary to clean and improve the data? 
- Did you create or modify any features to improve performance?

## Section 3. Feature Selection and Justification
### 3.1 Choose features and target
**Input features:**
- `TODO`, `TODO`, `TODO`

**Target variable:**
- `?` 
**Justification:**
- I selected these features because XYZ.


## 3.2 Define X and y
- Assign input features to X
- Assign target variable to y


In [8]:
# TODO

**Reflection 3:**
- Why did you choose these features? 
- How might they impact predictions or accuracy?

## Section 4. Train a Model (Linear Regression)
### 4.1 Split the data into training and test sets using train_test_split (or StratifiedShuffleSplit if class imbalance is an issue).

In [9]:
# TODO

### 4.2 Train model using Scikit-Learn model.fit() method

In [10]:
# TODO

### 4.3 Evalulate performance, for example:
- Regression: R^2, MAE, RMSE (RMSE has been recently updated)
- Classification: Accuracy, Precision, Recall, F1-score, Confusion Matrix
- Clustering: Inertia, Silhouette Score

In [11]:
# TODO

**Reflection 4:**
- How well did the model perform? 
- Any surprises in the results?

## Section 5. Improve the Model or Try Alternates (Implement Pipelines)
### 5.1 Implement Pipeline 1: Imputer → StandardScaler → Linear Regression.

In [12]:
# TODO

### 5.2 Implement Pipeline 2: Imputer → Polynomial Features (degree=3) → StandardScaler → Linear Regression.

In [13]:
# TODO

### 5.3 Compare performance of all models across the same performance metrics

In [14]:
# TODO

**Reflection 5:**
- Which models performed better? 
- How does scaling impact results?

## Section 6. Final Thoughts & Insights
### 6.1 Summarize findings.

### 6.2 Discuss challenges faced.

### 6.3 If you had more time, what would you try next?

**Reflection 6:**
- What did you learn from this project?